# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "weather_data/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Albany,42.6001,-73.9662,62.28,76,97,1.01,US,overcast clouds
1,1,Whittlesea,-32.1759,26.8244,69.80,41,57,11.05,ZA,broken clouds
2,2,Burshtyn,49.2587,24.6278,53.13,80,100,6.44,UA,light rain
3,3,Thompson,55.7435,-97.8558,48.36,53,75,14.97,CA,broken clouds
4,4,Puerto Ayora,-0.7393,-90.3518,75.16,91,100,8.99,EC,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Albany,42.6001,-73.9662,62.28,76,97,1.01,US,overcast clouds
1,1,Whittlesea,-32.1759,26.8244,69.80,41,57,11.05,ZA,broken clouds
16,16,Vaini,-21.2000,-175.2000,69.96,100,75,4.61,TO,broken clouds
17,17,Rikitea,-23.1203,-134.9692,68.16,60,6,17.81,PF,clear sky
19,19,Souillac,-20.5167,57.5167,71.98,73,40,12.66,MU,scattered clouds
22,22,Port Elizabeth,-33.9180,25.5701,62.89,83,0,14.97,ZA,clear sky
28,28,Vao,-22.6667,167.4833,73.15,83,0,11.30,NC,clear sky
29,29,East London,-33.0153,27.9116,63.54,82,20,12.66,ZA,few clouds
30,30,Plettenberg Bay,-34.0527,23.3716,60.49,88,97,6.04,ZA,overcast clouds
38,38,Cape Town,-33.9258,18.4232,61.12,82,20,3.44,ZA,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna(how='all')

# Display sample data
clean_travel_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Albany,42.6001,-73.9662,62.28,76,97,1.01,US,overcast clouds
1,1,Whittlesea,-32.1759,26.8244,69.80,41,57,11.05,ZA,broken clouds
16,16,Vaini,-21.2000,-175.2000,69.96,100,75,4.61,TO,broken clouds
17,17,Rikitea,-23.1203,-134.9692,68.16,60,6,17.81,PF,clear sky
19,19,Souillac,-20.5167,57.5167,71.98,73,40,12.66,MU,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
671,671,Talara,-4.5772,-81.2719,70.56,68,0,20.71,PE,clear sky
672,672,Vila Velha,-20.3297,-40.2925,73.35,82,100,9.22,BR,overcast clouds
674,674,Zemio,5.0314,25.1361,72.45,90,97,3.53,CF,overcast clouds
675,675,Betsiamites,48.9413,-68.6460,61.20,66,57,10.96,CA,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.

hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng
0,Albany,US,62.28,overcast clouds,42.6001,-73.9662
1,Whittlesea,ZA,69.80,broken clouds,-32.1759,26.8244
16,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000
17,Rikitea,PF,68.16,clear sky,-23.1203,-134.9692
19,Souillac,MU,71.98,scattered clouds,-20.5167,57.5167
22,Port Elizabeth,ZA,62.89,clear sky,-33.9180,25.5701
28,Vao,NC,73.15,clear sky,-22.6667,167.4833
29,East London,ZA,63.54,few clouds,-33.0153,27.9116
30,Plettenberg Bay,ZA,60.49,overcast clouds,-34.0527,23.3716
38,Cape Town,ZA,61.12,few clouds,-33.9258,18.4232


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Albany,US,62.28,overcast clouds,42.6001,-73.9662,
1,Whittlesea,ZA,69.80,broken clouds,-32.1759,26.8244,
16,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,
17,Rikitea,PF,68.16,clear sky,-23.1203,-134.9692,
19,Souillac,MU,71.98,scattered clouds,-20.5167,57.5167,
...,...,...,...,...,...,...,...
671,Talara,PE,70.56,clear sky,-4.5772,-81.2719,
672,Vila Velha,BR,73.35,overcast clouds,-20.3297,-40.2925,
674,Zemio,CF,72.45,overcast clouds,5.0314,25.1361,
675,Betsiamites,CA,61.20,broken clouds,48.9413,-68.6460,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Albany,US,62.28,overcast clouds,42.6001,-73.9662,
1,Whittlesea,ZA,69.80,broken clouds,-32.1759,26.8244,Eddie's Nest B&B
16,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Rikitea,PF,68.16,clear sky,-23.1203,-134.9692,People ThankYou
19,Souillac,MU,71.98,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
...,...,...,...,...,...,...,...
671,Talara,PE,70.56,clear sky,-4.5772,-81.2719,Hostal Oro Negro
672,Vila Velha,BR,73.35,overcast clouds,-20.3297,-40.2925,Hotel Vitória Palace
674,Zemio,CF,72.45,overcast clouds,5.0314,25.1361,OCHA Office
675,Betsiamites,CA,61.20,broken clouds,48.9413,-68.6460,Pourvoirie du lac des îles


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Albany,US,62.28,overcast clouds,42.6001,-73.9662,
1,Whittlesea,ZA,69.80,broken clouds,-32.1759,26.8244,Eddie's Nest B&B
16,Vaini,TO,69.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Rikitea,PF,68.16,clear sky,-23.1203,-134.9692,People ThankYou
19,Souillac,MU,71.98,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
...,...,...,...,...,...,...,...
671,Talara,PE,70.56,clear sky,-4.5772,-81.2719,Hostal Oro Negro
672,Vila Velha,BR,73.35,overcast clouds,-20.3297,-40.2925,Hotel Vitória Palace
674,Zemio,CF,72.45,overcast clouds,5.0314,25.1361,OCHA Office
675,Betsiamites,CA,61.20,broken clouds,48.9413,-68.6460,Pourvoirie du lac des îles


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Albany</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 62.28 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Eddie's Nest B&B</dd>\n<dt>City</dt><dd>Whittlesea</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>broken clouds and 69.8 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Weather</dt><dd>broken clouds and 69.96 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>clear sky and 68.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Shanti Maurice Resort & Spa</dd>\n<dt>City</dt><dd>Souillac</dd>\n<dt>Country</dt><dd>MU</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 71.98 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>39 On Nile Guest House</dd>\n<dt>C

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,42.6001,-73.9662
1,-32.1759,26.8244
16,-21.2000,-175.2000
17,-23.1203,-134.9692
19,-20.5167,57.5167
22,-33.9180,25.5701
28,-22.6667,167.4833
29,-33.0153,27.9116
30,-34.0527,23.3716
38,-33.9258,18.4232


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
# YOUR CODE HERE

# Create a figure to add the Google map as a layer
# YOUR CODE HERE

# Display the figure containing the map
# YOUR CODE HERE
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))